# Prepare Glove vector

In [1]:
import numpy as np
import pickle
import operator

In [2]:
def loadGloveModel(gloveFile):
    print ("Loading Glove Model... will take time 4-5 mins")
    model = {}
    
    with open(gloveFile, 'r') as f:
        
        lines = f.readlines()
        
        for line in lines:
            splitLine = line.split(' ')
            word = splitLine[0]
            embedding = [float(val) for val in splitLine[1:]]
            model[word] = embedding
        print ("Done.",len(model)," words loaded!")
    
    return model

In [3]:
def cal_coverage(list_voca, glove):

    cnt = 0
    for i in range(len(list_voca)):
        token = list_voca[i]
        if token in glove:
            ;
        else:
            cnt = cnt + 1  

    print ('# missing token : ' + str(cnt))
    print ('coverage : ' + str( 1 - ( cnt/ float(len(list_voca)) ) ) )

In [4]:
def create_glove_embedding(list_voca, glove):

    list_glove_voca = []
    cnt = 0

    for i in range(len(list_voca)):

        token = list_voca[i]
        
        if token in glove:
            if token == '':
                print ('TMP code add PAD as 0s - not desired')
                list_glove_voca.append( np.zeros(300) )
            else:
                list_glove_voca.append( glove[token] )
        else:
            if token == '':
                print ('_PAD_ embedding will be added in the model code as 0s')
                list_glove_voca.append( np.zeros(300) )  
            else:
                list_glove_voca.append( np.random.uniform(-0.25, 0.25, 300).tolist() )
                cnt = cnt + 1  

    print ('coverage : ' + str( 1.0 - ( cnt / float(len(list_voca)) ) ))
    return list_glove_voca

## 01 load glove model

In [5]:
glove = loadGloveModel('../data/embedding/glove.840B.300d.txt')

Loading Glove Model... will take time 4-5 mins
('Done.', 2196016, ' words loaded!')


## 02 extract pre-trained glove vector that is matched to the index of the target corpus

In [6]:
list_voca = []
with open('../data/nela-17/whole/dic_mincutN.txt', 'rb') as f:
    list_voca = f.readlines()
    list_voca = [x.strip() for x in list_voca]
    
print ('total dic size : ' + str(len(list_voca)))
cal_coverage(list_voca, glove)

total dic size : 150318
# missing token : 40737
coverage : 0.728994531593


In [7]:
list_glove_voca = create_glove_embedding(list_voca, glove)
len(list_glove_voca)
np_glove = np.asarray(list_glove_voca, dtype=np.float32)
print (np.shape(np_glove))

_PAD_ embedding will be added in the model code as 0s
coverage : 0.729001184156
(150318, 300)


In [8]:
np.save('../data/nela-17/whole/W_embedding.npy', np_glove)